In [3]:
import pandas as pd
import re
import os
import sqlite3

In [4]:
conn = sqlite3.connect("db.sqlite3") # practice.db 연결
c = conn.cursor() # db를 사용하기 위한 cursor

In [5]:
df = pd.read_csv('daum_new.csv' , names=['no','title','date','url', 'content'])
df.to_csv('daum_news.csv')

In [6]:
def preprocessing_articles(data):
    contents = []
    contents_new = []

    for content in data['content']:

        content = content.split('\n')
        content = list(filter(lambda s: len(s) > 3, content))
        
        # 글 마지막에 항상 기자의 메일 주소가 추가되어 있다. 그 이후로 나오는 글은 불필요하므로 삭제함
        for i in range(1, len(content)):
            if '@' in content[-i]:
                break
        if i == len(content)-1:
            i = 1
        content_prep = content[:-i]

        # 너무 짧거나, 광고, copyright, 사진설명, 메일 등 불필요한 단어가 포함되어 있을 경우 불필요한 문장으로 판단하고 삭제 처리
        content_prep = list(filter(lambda s: len(s) > 30 and \
            '▶' not in s and \
                '©' not in s and \
                    '▲' not in s and \
                        '사진' not in s and \
                            '@' not in s and \
                                not re.findall("기자 *$", s) and \
                                    not re.findall("제공 *$", s) and \
                                        not re.findall("기자 *= *", s) , content_prep))
        
        # 언론사명이 들어간 prefix 삭제
        content_prep = list(map(lambda s: re.sub("^\[.*\]", "", s), content_prep)) #[스포츠서울]
        content_prep = list(map(lambda s: re.sub("^\{.*\}", "", s), content_prep)) #{뉴스}
        content_prep = list(map(lambda s: re.sub("^\(.*\)", "", s), content_prep)) #(서울=뉴시스1)
        content_prep = list(map(lambda s: re.sub("[DA:차트]", "", s), content_prep)) #(서울=뉴시스1)
        
        contents.append('\n'.join(content))
        contents_new.append('\n'.join(content_prep))

    data['content'] = contents
    data['content_prep'] = contents_new

    return data


if __name__ == "__main__":

    file = 'daum_news.csv'
    data = pd.read_csv(file)

    base_name, ext = os.path.splitext(os.path.basename(file))
    output_file = "{}_{}{}".format(base_name, "prep", ext)

    data = preprocessing_articles(data)
    data.to_csv(output_file, index=None)

In [7]:
df1 = pd.read_csv('daum_news_prep.csv')

In [8]:
df2 = df1[['title', 'date', 'url', 'content_prep']]
df2

,title,date,url,content_prep
0,"[S트리밍] 과르디올라 ""운 좋았다..린가드 클래스 있더라""",2021. 02. 28. 00:00,https://v.daum.net/v/20210228000010172,맨체스터 시티는 27일(한국시간) 영국 노스웨스잉글랜드지역 그레이터맨체스터주의 맨체...
1,다섯장 추혁진 오늘(28일) 생일..트롯픽 이벤트 주인공 [DA:차트],2021. 02. 28. 00:00,https://v.daum.net/v/20210228000009171,[] 다섯장 추혁진 오늘(28일) 생일…롯픽 이벤 주인공\n28...
2,"신혜선, 걸친 것만 '1227만원'..은은한 럭셔리룩 ""어디 거?""",2021. 02. 28. 00:00,https://v.daum.net/v/20210228000008169,'프라다' 신상 아이템 풀 착용한 배우 신혜선 \n한 손에 흰색 호보백을 든 신혜...
3,"정준영 ""유인석이 '선물 줄게' 하더니 성매매女 오더라"" [MD현장] (종합)",2021. 02. 28. 00:00,https://v.daum.net/v/20210228000007168,가수 정준영(32)이 푸른색 수의를 입고 법정에 들어서자 피고인 석에 앉아있던 군...
4,"에이스 동훈, 생일 맞아 팬들이 전하는 특별한 선물",2021. 02. 28. 00:00,https://v.daum.net/v/20210228000005167,"""동훈아, 생일 축하해"" 28일 그룹 에이스 동훈의 생일을 맞아 서울 마포구 합정역..."
...,...,...,...,...
565,"'봄을 닮은 미모' 정려원, SJSJ와 함께한 'Oh Tomorrow' MV",2021. 02. 28. 00:00,https://v.daum.net/v/20210228000002166,"배우 정려원이 다가오는 봄을 알렸다.지난 19일, 한섬(대표 김민덕)이 전개하는 영..."
566,[기후는 경제다]탈탄소가 대세..글로벌 '큰손'은 탄소기업 손절중,2021. 02. 28. 00:00,https://v.daum.net/v/20210228000001164,바이든정부 기대감..지난해 4Q ESG펀드액 29% 증가주식형액티브펀드 순유출에도 ...
567,"[S트리밍] 과르디올라 ""운 좋았다..린가드 클래스 있더라""",2021. 02. 28. 00:00,https://v.daum.net/v/20210228000010172,맨체스터 시티는 27일(한국시간) 영국 노스웨스잉글랜드지역 그레이터맨체스터주의 맨체...
568,다섯장 추혁진 오늘(28일) 생일..트롯픽 이벤트 주인공 [DA:차트],2021. 02. 28. 00:00,https://v.daum.net/v/20210228000009171,[] 다섯장 추혁진 오늘(28일) 생일…롯픽 이벤 주인공\n28...


In [9]:
df2.to_sql('news_content_prep', conn)

In [10]:
conn.close()